# Average the pilatus frames scan

This notbook can be used to average all pilatus frames in a timescan to obtain better statistics. 

The averaged frame can be saved as a `.tiff` image and/or an `.h5` file

The averaged frame are saved in `/data/visitors/danmax/PROPOSAL/VISIT/process/average/SAMPLE/`

In [ ]:
%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import tifffile
import DanMAX as DM
from datetime import datetime
from time import sleep

In [ ]:
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
fname = DM.findScan()

saveTiff = True # Save as .tiff image 
saveH5 = False # Save as .h5 file

dst = fname.replace('raw', 'process/average')
if not os.path.exists(os.path.dirname(dst)):
    print(f'Creating folder: {os.path.dirname(dst)}')
    os.makedirs(os.path.dirname(dst))
im = DM.getAverageImage(fname)

plt.figure()
plt.title(DM.getScan_id(fname))
plt.imshow(im,norm='log')
plt.colorbar(label='log(I)')

if saveTiff:
    tifffile.imwrite(dst.replace('.h5','_averaged.tiff'),
                     im[np.newaxis,:].astype(np.float32))
    print(f"Averaged frames saved as: {dst.replace('.h5','_averaged.tiff')}")

if saveH5:
    with h5py.File(dst.replace('.h5','_averaged.h5'),'w') as f:
        f.create_dataset('/entry/instrument/pilatus/data',
                         data=im[np.newaxis,:].astype(np.int32))
    print(f"Averaged frames saved as: {dst.replace('.h5','_average.h5')}")

## Automatic averaging of pilatus frames in timescans

This notebook will routinely look for new files with timescans and average the pilatus frames in each timescan.  

The averaged frame are saved as .tiff images in `/data/visitors/danmax/PROPOSAL/VISIT/process/average/SAMPLE/`  

Stop the script by pressing the `■ (interrupt the kernel)` button in the top tool bar. 


In [ ]:
automatic_averaging = False
scan_type = 'any' # timescan, dscan, ascan, meshct, etc.

#Create a distination folder for data if it does not exist
dst = os.getcwd().replace('raw', 'process/average')
if not os.path.exists(dst):
    print(f'Creating folder: {dst}')
    os.makedirs(dst)

while automatic_averaging:
    # find all scans of a specified scan type
    scans = DM.findAllScans(scan_type=scan_type, descending=False)
    # remove scan names with already existing average tiff images
    scans = [scan for scan in scans if not os.path.isfile(scan.replace('raw', 'process/average').replace('.h5','_averaged.tiff' ))]
    i=0
    for scan in scans:
        im = DM.getAverageImage(scan)
        tifffile.imwrite(dst.replace('.h5','_averaged.tiff'),
                        im[np.newaxis,:].astype(np.float32))
        i+=1
    print(f'{datetime.now()}'.split('.')[0],f' - {i:4d} integrated images',end='\r')
    sleep(5)